In [1]:
import pandas as pd
import numpy as np

## 第二章 Pandas基础

In [2]:
# 在读取 txt 文件时，经常遇到分隔符非空格的情况，read_table 有一个分割参数 sep ，
# 它使得用户可以自定 义分割符号，进行 txt 数据的读取
table1 = pd.read_table('ch2/my_table_special_sep.txt')   # 不使用sep时
print(table1)
table2 = pd.read_table('ch2/my_table_special_sep.txt', sep="\|\|\|\|", engine='python')  
# 注意使用转依字符，||||分隔，同时指定为python引擎，参数 sep 中使用的是正则表达式
print(table2)

              col1 |||| col2
0  TS |||| This is an apple.
1    GQ |||| My name is Bob.
2         WT |||| Well done!
  col1                 col2
0   TS    This is an apple.
1   GQ      My name is Bob.
2   WT           Well done!


### 基本函数操作

In [3]:
df = pd.read_csv('learn_pandas.csv')
df.info()      # dataframe简述

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   School       200 non-null    object 
 1   Grade        200 non-null    object 
 2   Name         200 non-null    object 
 3   Gender       200 non-null    object 
 4   Height       183 non-null    float64
 5   Weight       189 non-null    float64
 6   Transfer     188 non-null    object 
 7   Test_Number  200 non-null    int64  
 8   Test_Date    200 non-null    object 
 9   Time_Record  200 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 15.8+ KB


In [4]:
df.describe()     # 基本统计特征

,Height,Weight,Test_Number
count,183.000000,189.000000,200.000000
mean,163.218033,55.015873,1.645000
std,8.608879,12.824294,0.722207
min,145.400000,34.000000,1.000000
25%,157.150000,46.000000,1.000000
50%,161.900000,51.000000,1.500000
75%,167.500000,65.000000,2.000000
max,193.900000,89.000000,3.000000


In [5]:
print(df['School'].unique())     # 数据去重
print(df['School'].nunique())    # 去重后的个数统计
print(df['School'].value_counts())    # 统计每个值出现的次数

['A' 'B' 'C' 'D']
4
D    69
A    57
C    40
B    34
Name: School, dtype: int64


In [6]:
# 如果想要观察多个列组合的唯一值，可以使用 drop_duplicates
# 其中的关键参数是 keep ，默认值 first 表示每个组合保留第一次出现的所在行，
# last 表示保留最后一次出现的所在行，False 表示把所有重复组合所在的行剔除
df_demo = df[['Gender','Transfer','Name']]
df_demo.drop_duplicates(['Gender', 'Transfer'], keep='last')

,Gender,Transfer,Name
147,Male,NaN,Juan You
150,Male,Y,Chengpeng You
169,Female,Y,Chengquan Qin
194,Female,NaN,Yanmei Qian
197,Female,N,Chengqiang Chu
199,Male,N,Chunpeng Lv


In [7]:
## 替换函数
# 一般而言，替换操作是针对某一个列进行的，因此下面的例子都以Series举例。
# pandas中的替换函数可以归纳为三类：映射替换、逻辑替换、数值替换
df['Gender'].replace({'Female': 0, 'Male':1}).head()   # 女性为0，男性为1

0    0
1    1
2    1
3    0
4    1
Name: Gender, dtype: int64

In [8]:
# 逻辑替换包括了 where 和 mask ，这两个函数是完全对称的：
# where 函数在传入条件为 False 的对应行进行替换，而mask在传入条件为True的对应行进行替换，
# 当不指定替换值时，替换为缺失值
s = pd.Series([-1, 1.2345, 100, -50])
s.where(s<0,100)

0     -1.0
1    100.0
2    100.0
3    -50.0
dtype: float64

In [9]:
# s.round(2)   # 保留两位小数
# s.abs()      # 取绝对值
print(s.clip(0,2))    # 按上下界截断，超过下界的置为0，超过上界的置为2
print(s.clip(0,2).replace([0,2], [1,1]))    # 我还可以自定义超过上下界的赋值为多少

0    0.0000
1    1.2345
2    2.0000
3    0.0000
dtype: float64
0    1.0000
1    1.2345
2    1.0000
3    1.0000
dtype: float64


In [10]:
## 排序函数
## 排序共有两种方式，其一为值排序，其二为索引排序，对应的函数是 sort_values 和 sort_index 
df_demo =  df[['Grade', 'Name', 'Height', 'Weight']].set_index(['Grade','Name']) # 把这俩列设为索引
df_demo

Height  Weight
Grade     Name                          
Freshman  Gaopeng Yang     158.9    46.0
          Changqiang You   166.5    70.0
Senior    Mei Sun          188.9    89.0
Sophomore Xiaojuan Sun       NaN    41.0
          Gaojuan You      174.0    74.0
...                          ...     ...
Junior    Xiaojuan Sun     153.9    46.0
Senior    Li Zhao          160.9    50.0
          Chengqiang Chu   153.9    45.0
          Chengmei Shen    175.3    71.0
Sophomore Chunpeng Lv      155.7    51.0

[200 rows x 2 columns]

In [11]:
# 对身高进行排序，默认是升序排列
df_demo.sort_values('Height').head(5)

,,Height,Weight
Grade,Name,,
Junior,Xiaoli Chu,145.4,34.0
Senior,Gaomei Lv,147.3,34.0
Sophomore,Peng Han,147.8,34.0
Senior,Changli Lv,148.7,41.0
Sophomore,Changjuan You,150.5,40.0


In [12]:
# 在排序中，经常遇到多列排序的问题，比如在体重相同的情况下，
# 对身高进行排序，并且保持身高降序排列， 体重升序排列（这里是先排体重然后再排身高）
df_demo.sort_values(['Weight','Height'],ascending=[True,False]).head()

,,Height,Weight
Grade,Name,,
Sophomore,Peng Han,147.8,34.0
Senior,Gaomei Lv,147.3,34.0
Junior,Xiaoli Chu,145.4,34.0
Sophomore,Qiang Zhou,150.5,36.0
Freshman,Yanqiang Xu,152.4,38.0


In [13]:
# 按索引排序和前面方式一样，此时需要指定索引层的名字或者层号，用参数level表示
# 需要注意的是字符串的排列顺序由字母顺序决定
df_demo.sort_index(level=['Grade','Name'],ascending=[True,False]).head()

Height  Weight
Grade    Name                         
Freshman Yanquan Wang    163.5    55.0
         Yanqiang Xu     152.4    38.0
         Yanqiang Feng   162.3    51.0
         Yanpeng Lv        NaN    65.0
         Yanli Zhang     165.1    52.0

In [14]:
## apply函数，第一个参数为一个操作函数，可以自定义，第二个参数同样也是指定轴axis
df_demo.apply(lambda x:x.mean(), axis=1).head()
## 一般不使用apply的方式对数据集进行处理，性能较差

Grade      Name          
Freshman   Gaopeng Yang      102.45
           Changqiang You    118.25
Senior     Mei Sun           138.95
Sophomore  Xiaojuan Sun       41.00
           Gaojuan You       124.00
dtype: float64

### 窗口对象（处理序列信息常用）

In [15]:
# pandas中有3类窗口，分别是滑动窗口rolling、扩张窗口expanding以及指数加权窗口ewm

## 滑窗对象
## 在得到了滑窗对象后，能够使用相应的聚合函数进行计算，需要注意的是窗口包含当前行所在的元素
## 比如这里我们令窗口为3，计算均值，即为窗口内元素的均值，不指定方向的话都是从左往右滑动


seq = pd.Series([1,2,3,4,5,6])
roller = seq.rolling(window=3)
print(roller.mean())

## shift, diff, pct_change 是一组类滑窗函数，它们的公共参数为 periods=n ，默认为 1
## 向前取第n个元素的值，向前做差分运算，向前计算增长率
s = pd.Series([1,3,6,10,15])
print(s.shift(2))
print(s.diff(2))
# 这里的 n 可以为负，表示反方向的类似操作
print(s.shift(-1))

## 可以找到滑动窗口和上述类滑窗函数的等价关系
## 比如s.rolling(3).apply(lambda x:list(x)[0])等价于s.shift(2)

## 练习：rolling 对象的默认窗口方向都是向前的，某些情况下用户需要向后的窗口如何实现？



0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
5    5.0
dtype: float64
0    NaN
1    NaN
2    1.0
3    3.0
4    6.0
dtype: float64
0    NaN
1    NaN
2    5.0
3    7.0
4    9.0
dtype: float64
0     3.0
1     6.0
2    10.0
3    15.0
4     NaN
dtype: float64


In [16]:
# 扩张窗口又称累计窗口，可以理解为一个动态长度的窗口，其窗口的大小就是从序列开始处到具体操作的对应位置，
# 其使用的聚合函数会作用于这些逐步扩张的窗口上。具体地说，设序列为 a1, a2, a3, a4，
# 则其每个位置对应的窗口即 [a1]、[a1, a2]、[a1, a2, a3]、[a1, a2, a3, a4]

s = pd.Series([1, 3, 6, 10])
s.expanding().mean()

0    1.000000
1    2.000000
2    3.333333
3    5.000000
dtype: float64

## 第三章 索引

- 一些简单的loc，iloc等等的索引切片方法就不在这里赘述了

### 索引器

In [17]:
# 在 pandas中，支持把字符串形式的查询表达式传入query方法来查询数据，其表达式的执行结果必须返回布尔列表

df.query('(School=="A") &'' (Gender=="Male") &' 'Weight>60')   
# 比如这里把学校和性别为指定值且体重大于60的数据筛选出来，单引号里面还可以调用各种统计函数
# 同时，在query中还注册了若干英语的字面用法，帮助提高可读性，例如：or, and, or, is in, not in

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
2,A,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
10,A,Freshman,Xiaopeng Zhou,Male,174.1,74.0,N,1,2019/9/29,0:05:16
21,A,Senior,Xiaopeng Shen,Male,166.0,62.0,NaN,1,2020/1/2,0:04:54
23,A,Senior,Qiang Zheng,Male,183.9,87.0,N,1,2019/12/5,0:04:59
50,A,Junior,Xiaoli Wang,Male,171.4,70.0,N,3,2019/12/20,0:05:12
60,A,Freshman,Yanpeng Lv,Male,NaN,65.0,N,1,2019/11/17,0:04:13
71,A,Sophomore,Feng Han,Male,183.4,82.0,N,2,2019/10/25,0:05:10
117,A,Freshman,Chunli Zhao,Male,180.2,83.0,N,1,2020/1/7,0:04:33
134,A,Senior,Gaoli Zhao,Male,186.5,83.0,N,1,2019/9/7,0:04:14
153,A,Freshman,Changmei Lv,Male,172.2,75.0,N,1,2019/10/6,0:04:15


In [18]:
## 随机抽样
## 如果把 DataFrame 的每一行看作一个样本，或把每一列看作一个特征，再把整个 DataFrame 看作总体，
## 想要对样本或特征进行随机抽样就可以用 sample 函数

## sample 函数中的主要参数为 n, axis, frac, replace, weights ，前三个分别是指抽样数量
## 抽样的方向（0 为行、1 为列）和抽样比例（0.3 则为从总体中抽出 30% 的样本）

df_sample = pd.DataFrame({'id': list('abcde'),
                         'values': [1,2,3,4,90]})
df_sample.sample(frac=0.4, replace=False)   # weights可以设置得到采样值的概率

,id,values
3,d,4
0,a,1


### 多级索引

In [19]:
## 为了学习本节内容先构建一个表
np.random.seed(0)
multi_index = pd.MultiIndex.from_product([list('ABCD'),
df.Gender.unique()], names=('School', 'Gender'))

multi_column = pd.MultiIndex.from_product([['Height', 'Weight'],
df.Grade.unique()], names=('Indicator', 'Grade'))

df_multi = pd.DataFrame(np.c_[(np.random.randn(8,4)*5 + 163).tolist(), (np.random.randn(8,4)*5 + 65).tolist()],
                        index = multi_index,
                        columns = multi_column).round(1)
df_multi

Indicator       Height                           Weight                   \
Grade         Freshman Senior Sophomore Junior Freshman Senior Sophomore   
School Gender                                                              
A      Female    171.8  165.0     167.9  174.2     60.6   55.1      63.3   
       Male      172.3  158.1     167.8  162.2     71.2   71.0      63.1   
B      Female    162.5  165.1     163.7  170.3     59.8   57.9      56.5   
       Male      166.8  163.6     165.2  164.7     62.5   62.8      58.7   
C      Female    170.5  162.0     164.6  158.7     56.9   63.9      60.5   
       Male      150.2  166.3     167.3  159.3     62.4   59.1      64.9   
D      Female    174.3  155.7     163.2  162.1     65.3   66.5      61.8   
       Male      170.7  170.3     163.8  164.9     61.6   63.2      60.9   

Indicator             
Grade         Junior  
School Gender         
A      Female   65.8  
       Male     63.5  
B      Female   74.8  
       Male     68.9  
C      Female   66.9  
       Male     67.1  
D      Female   63.2  
       Male     56.4

In [20]:
# 对于上面这个多层的表
# 如果想要得到某一层的索引，则需要通过 get_level_values 获得
df_multi.index.get_level_values(0)

Index(['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D'], dtype='object', name='School')

In [21]:
df_multi = df.set_index(['School', 'Grade'])
# 学校和年级设为索引，此时的行为多级索引，列为单级索引
df_multi.head(5)

Name  Gender  Height  Weight Transfer  \
School Grade                                                        
A      Freshman     Gaopeng Yang  Female   158.9    46.0        N   
B      Freshman   Changqiang You    Male   166.5    70.0        N   
A      Senior            Mei Sun    Male   188.9    89.0        N   
C      Sophomore    Xiaojuan Sun  Female     NaN    41.0        N   
       Sophomore     Gaojuan You    Male   174.0    74.0        N   

                  Test_Number  Test_Date Time_Record  
School Grade                                          
A      Freshman             1  2019/10/5     0:04:34  
B      Freshman             1   2019/9/4     0:04:20  
A      Senior               2  2019/9/12     0:05:22  
C      Sophomore            2   2020/1/3     0:04:08  
       Sophomore            2  2019/11/6     0:05:22

In [25]:
# 使用iloc和loc的方式与单级索引表几乎一致
df_multi = df_multi.sort_index()
df_multi.loc[('A', 'Junior')].head()

Name  Gender  Height  Weight Transfer  Test_Number  \
School Grade                                                               
A      Junior   Feng Zheng  Female   165.6    51.0        N            1   
       Junior    Mei Zhang  Female   156.5    44.0        N            1   
       Junior  Xiaoli Wang    Male   171.4    70.0        N            3   
       Junior     Qiang Lv  Female   152.1    42.0        N            2   
       Junior      Mei Sun  Female   159.5    50.0        N            1   

                Test_Date Time_Record  
School Grade                           
A      Junior  2019/12/20     0:05:23  
       Junior   2019/9/13     0:04:38  
       Junior  2019/12/20     0:05:12  
       Junior   2019/11/3     0:05:21  
       Junior  2019/11/22     0:05:20

## 第四章 分组

### 分组的模式及其对象

In [28]:
# 要实现分组操作，必须明确三个要素：分组依据、数据来源、操作及其返回结果
# 因此分组的一般模式为df.groupby(分组依据)[数据来源].使用操作
# 例如这里我们想返回学生体测的数据按性别统计身高的中位数
df.groupby('Gender')['Height'].median()

Gender
Female    159.6
Male      173.4
Name: Height, dtype: float64

In [31]:
# 现在需要根据多个维度进行分组,事实上，只需在 groupby 中传入相应列名构成的列表即可
print(df.groupby(['School','Gender'])['Height'].mean())
# 则会分别统计不同学校不同性别的身高平均值


# 还可以指定先指定好分组条件，比如体重大于整体均值的分组，同样还是计算身高的均值
condition = df.Weight > df.Weight.mean()
print(df.groupby(condition)['Height'].mean())

School  Gender
A       Female    159.122500
        Male      176.760000
B       Female    158.666667
        Male      172.030000
C       Female    158.776923
        Male      174.212500
D       Female    159.753333
        Male      171.638889
Name: Height, dtype: float64
Weight
False    159.034646
True     172.705357
Name: Height, dtype: float64


In [38]:
## 关于groupby对象的一些属性和方法
gb = df.groupby(['School', 'Grade'])   # 调用groupby实际上是生成一个对象，它有很多属性
gb.ngroups           # 可以返回分了多少组
res = gb.groups
print(res.keys())    # 可以返回每组对应的索引字典
gb.size()     # 返回每一组下面对应的元素个数

# 通过 get_group 方法可以直接获取所在组对应的行，此时必须知道组的具体名字
gb.get_group(('A', 'Freshman')).iloc[:3,:3]

dict_keys([('A', 'Freshman'), ('A', 'Junior'), ('A', 'Senior'), ('A', 'Sophomore'), ('B', 'Freshman'), ('B', 'Junior'), ('B', 'Senior'), ('B', 'Sophomore'), ('C', 'Freshman'), ('C', 'Junior'), ('C', 'Senior'), ('C', 'Sophomore'), ('D', 'Freshman'), ('D', 'Junior'), ('D', 'Senior'), ('D', 'Sophomore')])


,School,Grade,Name
0,A,Freshman,Gaopeng Yang
6,A,Freshman,Qiang Chu
10,A,Freshman,Xiaopeng Zhou


### 聚合函数

In [39]:
# 本身自带的聚合函数有    max/min/mean/median/count/all/any
# /idxmax/idxmin/mad/nunique/skew/quantile/sum/std/var/sem/size/prod
# 这些聚合函数当传入的数据来源包含多个列时，将按照列进行迭代计算：
df.groupby('Gender')[['Height', 'Weight']].max()

,Height,Weight
Gender,,
Female,170.2,63.0
Male,193.9,89.0


In [44]:
'''
虽然在 groupby 对象上定义了许多方便的函数，但仍然有以下不便之处：
• 无法同时使用多个函数
• 无法对特定的列使用特定的聚合函数
• 无法使用自定义的聚合函数
• 无法直接对结果的列名在聚合前进行自定义命名
'''
# 因此就需要agg函数来解决这些问题
# 【a】使用多个聚合函数
gb = df.groupby('Gender')[['Height', 'Weight']]
gb.agg(['sum', 'idxmax', 'skew'])   # idxmax是返回指定轴分组中最大值所在的index，skew是求偏度

# 【b】对特定的列使用特定的聚合函数
# 通过构造字典传入agg函数实现，其中键为列名，值为聚合方法
gb.agg({'Height':['max', 'mean'], 'Weight': 'count'})

# 【c】使用自定义函数，这里计算极差
gb.agg(lambda x: x.mean()-x.min())

# 【d】聚合结果重命名
# 如果想要对结果进行重命名，只需要将上述函数的位置改写成元组，
# 元组的第一个元素为新的名字，第二个位置为原来的函数
gb.agg([('range', lambda x: x.max()-x.min()), ('my_sum', 'sum')])

Height          Weight        
        range   my_sum  range  my_sum
Gender                               
Female   24.8  21014.0   29.0  6469.0
Male     38.2   8854.9   38.0  3929.0

### 变换和过滤（挺重要的）

In [45]:
# 变 换 函 数 的 返 回 值 为 同 长 度 的 序 列， 最 常 用 的 内 置 变 换 函 数 是 累 计 函 数：
# cum-count/cumsum/cumprod/cummax/cummin ，它们的使用方式和聚合函数类似，只不过完成的是组内累计操作
gb.cummax()

,Height,Weight
0,158.9,46.0
1,166.5,70.0
2,188.9,89.0
3,NaN,46.0
4,188.9,89.0
...,...,...
195,170.2,63.0
196,170.2,63.0
197,170.2,63.0
198,193.9,89.0


In [47]:
# 用自定义变换时需要使用 transform 方法，被调用的自定义函数，其传入值为数据源的序列，
# 最后的返回结果是行列索引与数据源一致的 DataFrame
gb.transform(lambda x: (x-x.mean())/x.std())   # 这里显然就是对身高体重进行了标准化操作

,Height,Weight
0,-0.058760,-0.354888
1,-1.010925,-0.355000
2,2.167063,2.089498
3,NaN,-1.279789
4,0.053133,0.159631
...,...,...
195,-1.048078,-0.354888
196,0.336968,0.385033
197,-1.048078,-0.539868
198,0.237570,-0.226342


In [53]:
# 过滤就是筛选分组的过程
# 组过滤作为行过滤的推广，指的是如果对一个组的全体所在行进行统计的结果返回 True 则会被保留，
# False则该组会被过滤，最后把所有未被过滤的组其对应的所在行拼接起来作为 DataFrame 返回
# 在groupby 对象中，定义了filter方法进行组的筛选，传入参数为自定义的函数，同时一定要保证函数返回为布尔值
gb.filter(lambda x: x.shape[0] > 100)


,Height,Weight
0,158.9,46.0
3,NaN,41.0
5,158.0,51.0
6,162.5,52.0
7,161.9,50.0
...,...,...
191,166.6,54.0
194,160.3,49.0
195,153.9,46.0
196,160.9,50.0


### 跨列分组

In [54]:
# 有时我们需要在不同分组中计算，每组返回一个标量，此时用上述任何一种函数都无法完成：比如计算BMI指数
# 此时则需要引入apply函数来进行跨列分组
# apply函数的使用和filter几乎完全一致，只是filter要求返回值必须是布尔类型

# 首先定义BMI函数
def BMI(x):
    Height = x['Height']/100
    Weight = x['Weight']
    BMI_value = Weight/Height**2
    return BMI_value.mean()

gb.apply(BMI)

Gender
Female    18.860930
Male      24.318654
dtype: float64

In [74]:
# 练习：处理汽车数据集，Brand, Disp., HP 分别代表汽车品牌、发动机蓄量、发动机输出
car = pd.read_csv("car.csv")
# 把country中出现两次以上的组保留下来
temp = car.groupby('Country').filter(lambda x: x.shape[0] > 2)
# 再按 Country 分组计算价格均值、价格变异系数、该 Country 的汽车数量，
# 其中变异系数的计算方法是标准差除以均值，并在结果中把变异系数重命名为 CoV
temp.groupby('Country')['Price'].agg(['mean', ('CoV', lambda x:x.std()/x.mean()), 'count'])

,mean,CoV,count
Country,,,
Japan,13938.052632,0.387429,19
Japan/USA,10067.571429,0.240040,7
Korea,7857.333333,0.243435,3
USA,12543.269231,0.203344,26


### 第七章 处理缺失数据